In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

## Loading and preparing label maps

In [4]:
with open('../../temp/all_labels_names_300.pkl', 'rb') as fin:
    mid2name_all = pickle.load(fin)

In [5]:
with open('../../temp/all_labels_300+.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [6]:
with open('../../results/det_results_concat_20.pkl', 'rb') as fin:
    det_results_concat = pickle.load(fin)

In [7]:
with open('../../results/det_results_merged_20b.pkl', 'rb') as fin:
    det_results_merged = pickle.load(fin)

In [8]:
with open('../../results/det_results_postproc_20b.pkl', 'rb') as fin:
    det_results_postproc = pickle.load(fin)

## Choosing examples for each class to visualize

In [9]:
label_to_img = {key: [] for key in all_labels}
for key, val in det_results_concat.items():
    for det in val:
        if det['score'] >= 0.1:
            label_to_img[det['label']].append(key)

In [10]:
sort_idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [11]:
for i in sort_idx:
    print(all_labels[i], mid2name_all[all_labels[i]], len(label_to_img[all_labels[i]]))

/m/01g317 Person 135341
/m/04yx4 Man 29490
/m/09j2d Clothing 24133
/m/0dzct Human face 19644
/m/0k4j Car 10452
/m/07yv9 Vehicle 8572
/m/03bt1vf Woman 7897
/m/07j7r Tree 5968
/m/01prls Land vehicle 5624
/m/0cgh4 Building 5578
/m/03q69 Hair 5563
/m/04hgtk Human head 5274
/m/01n5jq Poster 5150
/m/01rkbr Tie 4191
/m/01xyhv Suit 4170
/m/0k0pj Nose 3984
/m/0cmf2 Airplane 3771
/m/0k65p Hand 3055
/m/083kb Weapon 2836
/m/05r655 Girl 2584
/m/07r04 Truck 2522
/m/083wq Wheel 2470
/m/019jd Boat 2449
/m/01mzpv Chair 2420
/m/0dzf4 Arm 2203
/m/03jm5 House 2125
/m/05s2s Plant 2121
/m/09j5n Footwear 2003
/m/03120 Flag 1867
/m/0bt_c3 Book 1823
/m/014sv8 Eye 1806
/m/04bcr3 Table 1785
/m/0d4v4 Window 1744
/m/04dr76w Bottle 1710
/m/012t_z Businessperson 1695
/m/0k5j Aircraft 1685
/m/01xr66 Spokesperson 1665
/m/07jdr Train 1648
/m/07cmd Tank 1627
/m/0fly7 Jeans 1518
/m/04rky Mammal 1462
/m/080hkjn Handbag 1403
/m/0c_jw Furniture 1355
/m/09x0r Speech 1347
/m/0h9mv Tire 1325
/m/07s6nbt Text 1297
/m/06c54 Rifle

/m/0176mf Belt 0
/m/063rgb Mixer 0
/m/084rd Wok 0
/m/05n4y Ostrich 0
/m/06c7f7 Lipstick 0
/m/076lb9 Bench 0
/m/021mn Cookie 0
/m/06ncr Saxophone 0
/m/03s_tn Kettle 0
/m/01xs3r Jet ski 0
/m/03ldnb Ceiling fan 0
/m/015x4r Cucumber 0
/m/02y6n French fries 0
/m/03fwl Goat 0
/m/02w3_ws Personal care 0
/m/0323sq Golf cart 0
/m/01dwwc Pancake 0
/m/026qbn5 studio couch 0
/m/02zvsm Tart 0
/m/0cydv Caterpillar 0
/m/0642b4 Cupboard 0
/m/033cnk Egg 0
/m/03fj2 Goldfish 0
/m/0cl4p Hedgehog 0
/m/03kt2w Stationary bicycle 0
/m/0cn6p Otter 0
/m/01_5g Chopsticks 0
/m/06mf6 Rabbit 0
/m/09k_b Lemon 0
/m/01hrv5 Popcorn 0
/m/01krhy Tiara 0
/m/0mw_6 Willow 0
/m/0703r8 Loveseat 0
/m/02vkqh8 Wardrobe 0
/m/0h8n6f9 Dog bed 0
/m/03m5k Harp 0
/m/0d4w1 Closet 0
/m/03bj1 Panda 0
/m/02w3r3 Paper towel 0
/m/07clx Tea 0
/m/02cvgx Winter melon 0
/m/0bjyj5 Waste container 0
/m/0pcr Alpaca 0
/m/0ccs93 Canary 0
/m/0cjq5 Lobster 0
/m/0kpqd Watermelon 0
/m/03jbxj Light switch 0
/m/02wv84t Gas stove 0
/m/03v5tg Drinking straw

In [12]:
#select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3', '/m/0dwx7']


In [13]:
with open('../../../wsod/metadata/ont_300/300-classes.csv', 'r') as fin:
    select_labels = [item.strip() for item in fin]


## Visualizing Results

In [14]:
mid2name_extended = {}
extended_classes = set()
for key in all_labels:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('pascal','PA'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J'), ('pascal/J','PA/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_all[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [15]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [16]:
result_dict = {
    'Not Merged': det_results_concat,
    'Merged': det_results_merged,
    'Post-Processed': det_results_postproc,
}

In [17]:
path_prefix = '../../../../data/eval_m9/jpg/' 

In [18]:
model_abv_dict = {
    'coco': 'CO',
    'pascal': 'PA',
    'oi': 'OI',
    'ws': 'WS',
    'coco/J': 'CO/J',
    'pascal/J': 'PA/J',
    'oi/J': 'OI/J',
    'ws/J': 'WS/J',
}

In [22]:
def show(imgid, source, thresh):
    filename = path_prefix + imgid + '.jpg.ldcc'
    print(imgid)
    with open(filename, 'rb') as fin:
        _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    print(imgbgr.shape)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = result_dict[source][imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx_extended[det['label'] + '/' + det['model']] for det in detections]
    
    # print([(mid2name_extended[extended_classes[item]], boxes[i]) for i, item in enumerate(label_idx) if scores[i] > thresh])
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index_extended,
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      max_boxes_to_draw=1000,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [36]:
imgid = 'IC0015LAC'
w = interactive(show, 
    imgid=fixed(imgid),
    source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
    thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
)
w.children[-1].layout.height = '600px'
display(w)
    

interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…